# Changelog
This is a written documentation of big bugs I come across, what I did to solve them, and comments moving forward.

### Fixes for Dimensionality Reduction in Large Datasets
Date: 2025 -09

#### Summary:
Stabilized dimensionality reduction pipeline for both small (100-node) and large (700-node) datasets. Eliminated degenerate “all zeros” reductions and fixed optimizer stalls in higher-order correlation levels.


timecorr/helpers.py || weighted_timepoint_decoder -> optimize_weights
* Passed in a opt_init value of random, this gives a random starting point, to help with getting stuck on the first iteration and not progressing further. (as gradient descent too flat)
* Inside of minimize fucntion, added method='SLSQP'. This matches the original implementation is scipy 1.2.1. It has obvously been awhile since then and this is necessary as the minimize function has been updated in the recent years.


Changes
timecorr/braintools/brain_reduce.py || reduce
* replaced "Rows <= ndims: returning zeros." behavior with a clamp:
if n_comp is not None and rows <= n_comp:
    model_params['n_components'] = max(1, rows - 1)
Ensures reductions always return meaningful features, even when rows < ndims.
timecorr/helpers.py → reduce

* padding/truncation now handles list of arrays and ndarrays separately.
Prevents ValueError: all input arrays must have same number of dimensions during np.hstack.
Guarantees reduced outputs are always shaped (rows, V) per timepoint.
timecorr/helpers.py → weighted_timepoint_decoder

* Impact:
Higher-order correlation decoding now stable for both 100-node and 700-node dataset, and optimization no longer collapses to uniform weights due to zeroed features.


### Fixes for decresing accuracy as levels went up
Date: 2025 -11

#### Summary:
Stabilized gradient descent so that you will not get stuck on the wrong gradient descent path. Previously, the accuracys as you increase higher order correlation would sometimes decrease. But this should not be the case because we should find the optimal weights even if that is the same weights as the level before.


timecorr/helpers.py || weighted_timepoint_decoder
Changes:
Track best_acc, best_mu_vec, best_row across optimize_levels.
Accept if accuracy ≥ best; otherwise record prior best and zero newest weight.
Add _mu_as_vector() to normalize optimizer output (fixes KeyError: np.int64(0)).
Add _pad_mu() to align vector lengths (fixes IndexError on rejections).

Behavior:
Accuracy per fold no longer decreases with level; mean across folds also non-decreasing.
When higher levels don’t help, their weights become 0, effectively reverting to the best lower level.

Impact:
More stable curves; same schema (weights level_0..level_k), with some newest weights recorded as 0.0.



* Impact:

